# The quantum approximate optimization algorithm (QAOA)

<div class="alert alert-warning">
    Got feedback? Share your thoughts by filling out our <a href="https://forms.gle/z6ochGoC6grMh65E9">survey</a>.
</div>

The quantum approximate optimization algorithm is a general method for tackling optimization problems using quantum computers. It closely resembles the [adiabatic quantum algorithm](https://en.wikipedia.org/wiki/Adiabatic_quantum_computation). The first step is to encode the problem as finding the ground state of an appropriate _cost Hamiltonian_ $H_C$ that is diagonal in the computational basis. Starting from an easy-to-prepare ground state of an initial Hamiltonian $H_M$, the algorithm evolves the initial state under the time-dependent Hamiltonian

$$
H(t) = (1-t/T)H_M + (t/T)H_C
$$

for some choice of $T$. If $T$ is chosen to be large enough such that the Hamiltonian changes very slowly compared to the energy scale of the system, the ground state of the initial Hamiltonian will move adiabatically to the ground state of the time-evolved Hamiltonian, ending in the ground state of the cost Hamiltonian and therefore in a solution to the problem. 

Quantum annealing is a version of this algorithm where the Hamiltonian changes rapidly in time, so there is no guarantee that the system will remain in a true ground state. The quantum approximate optimization algorithm (QAOA) is a take on this method where time evolution is approximated using a Trotter-Suzuki decomposition. Instead of evolving smoothly and slowly, the time of evolution is optimized at each step. 

<img src="fig/optimization_algorithms.png" style="width: 700px;"/>

Given a Hamiltonian $H$ and a time parameter $t$, we define the time evolution operator as:

$$
U(H, \ t) \ = \ e^{-i H t / \hbar}.
$$

In general, implementing a quantum circuit that exactly exponentiates a Hamiltonian with many non-commuting terms is very challenging.

Instead, we *Trotterize* and divide the entire time of evolution into $N$ intervals:

$$
U = \prod_{k=0}^Ne^{-i (1-k/N) H_M}e^{-i (k/N) H_C}.
$$
If $N$ is very large, this can be a good approximation which leads to a performant quantum algorithm. What QAOA does is to consider also the case where $N$ is more moderately sized and instead of applying time evolution for pre-determined parameters, we incorporate free parameters into each term of the expansion, leading to a parametrized sequence of gates

$$
U(\gamma, \alpha) = e^{-i \alpha_1 H_M}
             e^{-i \gamma_1 H_C} \ ... \ e^{-i \alpha_N H_M} e^{-i \gamma_N H_C}.
$$

<img src="fig/qaoa_circuit.png" width=1000>



Once the optimization problem has been encoded into a diagonal cost Hamiltonian $H_C$, QAOA simply chooses an adequate *mixer Hamiltonian* $H_M$ and a number of layers $N$ and optimizes the parameters $\gamma, \alpha$ with respect to the cost function 

$$
\langle \psi| U(\gamma, \alpha)^\dagger H_C U(\gamma, \alpha)|\psi\rangle,
$$

where $|\psi\rangle$ is an initial state of choice.


## The PennLane QAOA module

PennyLane's [QAOA module](https://pennylane.readthedocs.io/en/stable/code/qml_qaoa.html) offers functionality to help in the construction of these Hamiltonians and circuits, to save you having to code everything up from scratch. 
It allows you to construct the cost Hamiltonians for many graph optimization problems directly from an input graph. It also provides recommended mixer Hamiltonians.

### The problem

Suppose you've successfully recast your favourite optimization problem into a `MaxIndependentSet` problem.

<img src="https://mathworld.wolfram.com/images/eps-gif/MaximumIndependentSet_1000.gif">
(Image source: Wolfram MathWorld)

The solution to this problem is encoded into bit strings. Each bit corresponds to a node, and the value is 1 if the node is in the max independent set, and 0 otherwise.

In [ ]:
import networkx as nx
from matplotlib import pyplot as plt

num_nodes = 6

G = nx.wheel_graph(num_nodes)

In [ ]:
nx.draw(G, with_labels=True, node_color='pink')

### Cost and mixer Hamiltonians

We can use built-in functionality in PennyLane to construct the cost Hamiltonian for the `MaxIndependentSet` problem defined on this graph:

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qaoa

In [ ]:
h_cost, h_mixer = qaoa.max_independent_set(G)
print(f'H_C = {h_cost}\n')
print(f'H_M = {h_mixer}')

### Building the QAOA circuit

The QAOA module gives you simple access to building the unitaries $e^{-i \gamma H_C}$ and $e^{-i \alpha H_M}$ using respectively the [`cost_layer()`](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.qaoa.layers.cost_layer.html) and [`mixer_layer()`](https://pennylane.readthedocs.io/en/stable/code/api/pennylane.qaoa.layers.mixer_layer.html) functions. For example, these can be used to create a circuit applying the QAOA unitary for $N=1$:

In [ ]:
def qaoa_layer(gamma, alpha):
    qaoa.cost_layer(gamma, h_cost)
    qaoa.mixer_layer(alpha, h_mixer)

To repeatedly apply this circuit, which corresponds to the case $N>1$, we can employ the `qml.layer()` function. In this example, we choose an initial state corresponding to a uniform superposition over all computational basis states and use a total of four layers, which can be optimized using the `ExpvalCost()` function:

In [ ]:
depth = 4

dev = qml.device('default.qubit', wires=range(num_nodes))

def circuit(params, **kwargs):
    qml.layer(qaoa_layer, depth, params[0], params[1])

cost = qml.ExpvalCost(circuit, h_cost, dev)   

In [ ]:
params = 0.5 * np.ones((2, 4))

num_iterations = 20

opt = qml.GradientDescentOptimizer(stepsize=0.4)

In [ ]:
for _ in range(num_iterations):
    params = opt.step(cost, params)

Once the circuit parameters have been optimized, the output state will be such that the probability of observing low-energy states is high, ideally having the ground state correspond to the most likely output state. In the algorithm, we measure the output state in the computational basis and record each outcome. The final output of the algorithm is the sampled state with the lowest energy. For the final exercise, we work with the graph defined below.

In [ ]:
@qml.qnode(dev)
def probability(p):
    circuit(p)
    return qml.probs(wires=dev.wires)

probs = probability(params)
plt.bar(range(2 ** num_nodes), probs)

In [ ]:
np.argmax(probs)

In [ ]:
np.binary_repr(20, 6)

In [ ]:
node_colour_vals = [int(x) for x in list(np.binary_repr(20, 6))]
node_colours = ['pink' if x == 0 else 'lightblue' for x in node_colour_vals]

In [ ]:
nx.draw(G, with_labels=True, node_color=node_colours)

## More things to try 

 - learn about QAOA in more detail with our [introductory demo](https://pennylane.ai/qml/demos/tutorial_qaoa_intro.html)
 - explore how the number of layers affects the results
 - try a different underlying graph problem (full list available [here](https://pennylane.readthedocs.io/en/stable/code/qml_qaoa.html#id1))